## 1. Import Python libraries 

In [ ]:
import os
import json
import h5py
import math
import shap
import itertools

import numpy as np
import pandas as pd

from tensorflow import keras
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import model_from_json
from datetime import timedelta as td, datetime
from IPython.display import clear_output

## 2. Define global variables 

In [ ]:
lr_value = 0.0001 ## LEARNING RATE
no_of_runs = 3 ## NO. OF RANDOM INTIALIZATION RUNS
train_percent = 0.80 ## PERCENTAGE OF TRAINING DATASET
batch_size = [4,8,16] ## BATCH SIZES
number_of_epochs = [500] ## EPOCHS
COMBINATIONS_LIST = [batch_size,number_of_epochs]
COMBINATIONS = list(itertools.product(*COMBINATIONS_LIST))

continents_list = ['Asia','Africa','Europe','Global-R','North America','South America']
multi_time_steps = [1,3,5,7,9] ## NO. OF MULTI-TIME STEPS (DAYS)
MA_day_list = [3,5,7] ## NO. OF MOVING AVERAGE DAYS
targets_list = ['G','D'] ## TARGET PARAMETERS
crit_MAPE_score = 0.700 ## MAXIMUM ALLOWABLE MEAN ABSOLUTE PERCENTAGE ERROR (MAPE) SCORE (%)
tol_runs = 10 ## TOTAL NO. OF TOLERANCE RUNS
analysis_folder = '' ## YOUR DESIRED DATA LOCATION FOR RESULTS STORAGE

## 3. Import data 

In [ ]:
DEFINE_YOUR_OWN_HDF5_LOC = '' ## YOUR DEFINED FILE LOCATION STORING THE PROCESSED HDF5 DATA FILES 
data_path_location = DEFINE_YOUR_OWN_HDF5_LOC
list_h5_files = os.listdir(data_path_location)

## 4. Define deep neural network function

In [ ]:
def deep_nn(x_data_array, y_data_array, ass_x_array):
    
    layers_neurons = [int(x_train.shape[1]/2),
                      int(x_train.shape[1]/4),
                      int(x_train.shape[1]/8),
                      1,
                      3,
                      3,
                      y_train.shape[1]]
    
    first_input = keras.Input(shape=(x_data_array.shape[1], ))
    second_dense = keras.layers.Dense(layers_neurons[0],
                                     activation='relu')(first_input)
    third_dense = keras.layers.Dense(layers_neurons[1],
                                     activation='relu')(second_dense)
    fourth_dense = keras.layers.Dense(layers_neurons[2],
                                      activation='relu')(third_dense)
    fifth_dense = keras.layers.Dense(layers_neurons[3],
                                      activation='relu')(fourth_dense)
    merge_one = keras.layers.concatenate([second_dense,
                                          third_dense,
                                          fourth_dense,
                                          fifth_dense])
    
    ## Data assimilation component 
    second_input = keras.Input(shape=(ass_x_array.shape[1], ))
    merge_two = keras.layers.concatenate([merge_one,second_input])
    
    ## merging 1D VD-FCNN with data assimilation component
    dummy_output_1 = keras.layers.Dense(layers_neurons[-3], 
                                        activation='relu')(merge_two)
    dummy_output_2 = keras.layers.Dense(layers_neurons[-2], 
                                        activation='relu')(dummy_output_1)
    final_output = keras.layers.Dense(layers_neurons[-1], 
                                      activation='relu')(dummy_output_2)
    
    final_model = keras.Model(inputs=[first_input, second_input], 
                              outputs = final_output)

    opt = keras.optimizers.Adam(learning_rate = lr_value)
    final_model.compile(optimizer = opt,
                        loss = 'mean_squared_error')
    final_model.summary()
    
    return final_model

## 5. Model training and validation 

In [ ]:
DEFINE_YOUR_SUMMARY_FILES_LOC = '' ## YOUR DEFINED FILE LOCATION STORING SUMMARY DATA FILES
processed_path_location = DEFINE_YOUR_SUMMARY_FILES_LOC

for target_type in targets_list:
    final_analysis_path_location = data_path_location + '/' + 'With ' + target_type  + "assimilation"
    if not os.path.exists(final_analysis_path_location):
        os.mkdir(final_analysis_path_location)
    for continent in continents_list:
        
        targets_filename = continent + '_G_D_targets.csv'
        final_targets_path = processed_path_location + '/' + targets_filename
        data_targets = pd.read_csv(final_targets_path)
        DATES_DATA_TARGETS = list(data_targets['Date'])
        train_dates_list = DATES_DATA_TARGETS[:int(train_percent*len(DATES_DATA_TARGETS))]
        test_dates_list = DATES_DATA_TARGETS[int(train_percent*len(DATES_DATA_TARGETS)):]
        
        for multi_time_value in multi_time_steps:
            
            for MA_days in MA_day_list:

                h5_file_location = data_path_location + '/' + continent + '_' + str(multi_time_value) + '_day_multi_time_steps-' \
                                   + str(MA_days) + '_days_MA.h5'
                data_records_file = h5py.File(h5_file_location, 'r')
                x_array = data_records_file.get('X_array')
                y_array = data_records_file.get(target_type + '_array')
                ass_x_array = data_records_file.get('ASS' + '_' + target_type + '_array')

                x_scaler = StandardScaler()
                x_array = np.array(x_array).reshape(len(x_array),-1)
                scaled_x_array = x_scaler.fit_transform(x_array)

                x_train = scaled_x_array[:int(train_percent*len(scaled_x_array))]
                y_train = y_array[:int(train_percent*len(y_array))]

                x_test = scaled_x_array[int(train_percent*len(scaled_x_array)):]
                y_test = y_array[int(train_percent*len(y_array)):]

                sep_x_train = ass_x_array[:int(train_percent*len(ass_x_array))]
                sep_x_test = ass_x_array[int(train_percent*len(ass_x_array)):]

                h5_file_folder = final_analysis_path_location + '/' + continent + '_' + str(multi_time_value) \
                                 + '_day_multi_time_steps-' + str(MA_days) + '_days_MA'
                if not os.path.exists(h5_file_folder):
                    os.mkdir(h5_file_folder)

                directory_models = h5_file_folder + '/' + 'Saved_Models'
                if not os.path.exists(directory_models):
                    os.makedirs(directory_models)

                directory_results = h5_file_folder + '/' + 'Saved_Results'
                if not os.path.exists(directory_results):
                    os.makedirs(directory_results)

                directory_figures = h5_file_folder + '/' + 'Saved_Figures'
                if not os.path.exists(directory_figures):
                    os.makedirs(directory_figures)
                    
                directory_text_files = h5_file_folder + '/' + 'Saved_Texts'
                if not os.path.exists(directory_text_files):
                    os.makedirs(directory_text_files)

                train_val_df = pd.DataFrame()
                test_df = pd.DataFrame()
                training_loss_df = pd.DataFrame()
                validation_loss_df = pd.DataFrame()

                train_val_df['Dates'] = train_dates_list
                train_val_df['Actual'] = [item[0] for item in y_train]
                test_df['Dates'] = test_dates_list
                test_df['Actual'] = [item[0] for item in y_test]
                
                ## TO STORE INTERMEDIATE MAPE RESULT SCORES ##
                results_text_file_location = directory_text_files + '/' + 'Results_LOG.txt'
                
                f= open(results_text_file_location,"w+")
                with open(results_text_file_location, 'w') as f:
                    f.write('Commence analysis =D')
                    f.write('\n')
                
                for item in COMBINATIONS:
                    number_of_epochs = item[1]
                    bs = item[0]
                    
                    error_counter = 0
                    dummy_mape_tol_value = crit_MAPE_score
                    while True:
                        deep_model = deep_nn(x_train,
                                             y_train,
                                             sep_x_train)
                        history_model = deep_model.fit([x_train,
                                                        sep_x_train],
                                                       y_train,
                                                       epochs = number_of_epochs,
                                                       validation_split = 0.2,
                                                       batch_size = bs,
                                                       verbose = 2)

                        predictions_train = deep_model.predict([x_train,
                                                                sep_x_train])
                        predictions_test = deep_model.predict([x_test,
                                                               sep_x_test])

                        MAPE = mean_absolute_percentage_error([item[0] for item in y_test],
                                                              [item[0] for item in predictions_test])   
                        with open(results_text_file_location, 'a') as f:
                            result_lines = 'Batch size = ' + str(bs) + ', Epochs = ' + str(number_of_epochs) + ': MAPE = ' \
                                           + str(round(MAPE,3))
                            f.writelines(result_lines)
                            f.write('\n')
                        
                        ## TO INITIATE SHAP ANALYSIS FOR EXPLAINABLE DEEP LEARNING COMPONENT ##
                        if MAPE < dummy_mape_tol_value:
                            model_json = deep_model.to_json()
                            json_filename = 'Batch size = ' + str(bs) + ', Epochs = ' + str(number_of_epochs) + '.json'
                            with open(directory_models + '/' + json_filename, "w") as json_file:
                                json_file.write(model_json)
                            h5_filename = 'Batch size = ' + str(bs) + ', Epochs = ' + str(number_of_epochs) + '.h5'
                            deep_model.save_weights(directory_models + '/' + h5_filename)
                            explainer = shap.KernelExplainer(deep_model.predict,
                                                             x_train)
                            shap_values = explainer.shap_values(X_test,
                                                                nsamples=100) ## CHANGE SAMPLES ACCORDING DEPENDING ON
                                                                              ## TOTAL DATA INSTANCES AVAILABLE IN 
                                                                              ## TESTING DATASET                            
                            break
                            
                        else:
                            error_counter += 1
                            
                        if error_counter % tol_runs == 0:
                            dummy_mape_tol_value += 0.05

                        clear_output(wait=True)

                    training_loss = history_model.history['loss']
                    validation_loss = history_model.history['val_loss']
                    training_loss_list = [item for item in training_loss]
                    validation_loss_list = [item for item in validation_loss]
                    training_loss_df[str(number_of_epochs) + '_epochs_' + str(bs) + '_batch_size'] = training_loss_list
                    validation_loss_df[str(number_of_epochs) + '_epochs_' + str(bs) + '_batch_size'] = validation_loss_list

                    train_val_df[str(number_of_epochs) + '_epochs_' + str(bs) + '_batch_size'] = [item[0] for item in predictions_train]
                    test_df[str(number_of_epochs) + '_epochs_' + str(bs) + '_batch_size'] = [item[0] for item in predictions_test]

                training_loss_df.to_csv(directory_results + '/' + 'training_loss.csv',
                                        index = False)
                validation_loss_df.to_csv(directory_results + '/' + 'validation_loss.csv',
                                          index = False)
                train_val_df.to_csv(directory_results + '/' + 'train_val_predictions.csv',
                                    index = False)
                test_df.to_csv(directory_results + '/' + 'test_predictions.csv',
                               index = False)
                clear_output(wait=True)